# Exercices Séance 3

## Exercice 1 : écrire des fichiers

Faire le script complet permettant de récupérer les pages webs suivantes, et les stocker dans un dossier "pages" 

In [ ]:
urls = [
    "https://www.unil.ch/unil/fr/home.html",
    "http://cuso.ch",
    "https://www.unige.ch/",
    "https://www.u-bordeaux.fr/",
]

Je veux donc :

Pour chaque adresse (donc sans doute une boucle) :
- faire une requête sur l'url
- récupérer son contenu
- l'écrire dans un fichier
- lui même contenu dans un dossier "pages"

In [26]:
import requests
import os  # va nous aider à créer/écrire des choses sur notre ordi

# Créer le dossier "pages" si il n'existe pas
# on aurait pu le faire à la main, mais ici on automatise
os.makedirs("pages", exist_ok=True)

for url in urls:
    response = requests.get(url)

    # dark, j'explique dessous
    filename = url.replace("/", "_") + ".txt"

    # Utiliser le module os pour donner un chemin et nom de fichier
    with open(os.path.join("pages", filename), "w") as f:
        f.write(response.text)

**Pourquoi le `.replace` ?**

si je garde les `/` ça foire car c'est aussi comme ça que l'on indique un chemin de dossier.  
Je remplace donc par un truc qui pose pas de pb dans un nom de fichier : "`_`".  
C'est moche, c'est nul, mais ça marche ici.

**Autre possibilité,** plus stable, slugify : https://pypi.org/project/python-slugify/

In [ ]:
# %pip install python-slugify

In [ ]:
from slugify import slugify

slug = slugify("https://www.unil.ch/unil/fr/home.html")
slug

### "faites-mieux"  (JLM vibe)
Il y a toujours mieux… mais ça marche.  
Ici en réalité il faudrait gérer les exceptions, etc.  
Mais voilà.  
"it works"


----

## Exercice 2 : utiliser beautiful soup

- Installer la bibliothèque beautiful soup https://www.crummy.com/software/BeautifulSoup/
- Récupérer tous les liens de la page cuso avec la méthode find_all (un lien est une balise a)
- Ecrire dans un fichier tous les liens

**read the doc !**  
Regarder la doc : https://beautiful-soup-4.readthedocs.io/en/latest/#quick-start

In [ ]:
# %pip install beautifulsoup4

In [ ]:
# version "brute" de base
from bs4 import BeautifulSoup
import requests

# Récupérer la page
response = requests.get("http://cuso.ch")
soup = BeautifulSoup(response.content, "html.parser")

# Trouver toutes les balises <a>
liens = soup.find_all("a")

# Enregistrer ce truc moche dans un fichier
with open("liens_moches_cuso.txt", "w") as f:
    f.write(str(liens))  # ici je transforme en str (sinon le type ResultSet passe pas)

### Fini ? (1)

### READ THE DOC !!!!!!

> One common task is extracting all the URLs found within a page’s <a> tags:
>```python
>for link in soup.find_all('a'):
>    print(link.get('href'))
>```

In [ ]:
# En fait nos "liens" sont pas des liens
# ie c'est moche
liens[5]

In [ ]:
# on peut choper ce qui nous intéresse plus ici :
liens[5].get("href")

In [ ]:
# Améliorer notre code :

# Extraire les URLs (href)
for lien in liens:
    href = lien.get("href")
    print(href)

In [ ]:
# Fusionner les logiques :
# améliorer notre code de base et renvoyer le tout dans la sortie fichier

# importer
from bs4 import BeautifulSoup
import requests

# Récupérer la page
response = requests.get("http://cuso.ch")
soup = BeautifulSoup(response.content, "html.parser")

# Trouver toutes les balises <a>
liens = soup.find_all("a")

#######################
# ON MODIFIE ci-dessous
#######################

# Extraire les href des liens et les écrire dans un fichier
with open("liens_cuso.txt", "w") as f:
    for lien in liens:
        href = lien.get("href")  # cette fois je récupère bien le href
        f.write(href + "\n")  # je saute une ligne entre les éléments avec "\n"

----

### Fini ?? (2) (possiblement pour plus tard selon le timing)

In [ ]:
liens[0].get("href")

**kekoikest-ce ?**

auto référence : des liens qui ne sont pas des liens externes pointant vers d'autres sites, mais propres au site de la cuso.    
et l'idée c'est :  
url_de_base + suite_url (/ ; /en/ ; etc.)  

COOL !! je sais concaténer des chaines de caractères :  

ma_chaine + la_suite

In [ ]:
"http://cuso.ch" + "/en/"  # et je suis sur la version anglaise

COOL !

COOL ???? 

…………………

PAS COOL !!!!!!! : je ne veux pas ajouter "http://cuso.ch" devant les liens externes :

"http://cuso.ch" + "https://www.unifr.ch/" = cata

MAIS y a une solution pour ça : la fonction `urljoin` de urllib.parse

In [ ]:
from urllib.parse import urljoin

for lien in liens:
    href = lien.get("href")
    url_complet = urljoin("http://cuso.ch", href)
    print(url_complet)

In [ ]:
# On implémente pour avoir des vraies url

# importer
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

# Récupérer la page
response = requests.get("http://cuso.ch")
soup = BeautifulSoup(response.content, "html.parser")

# Trouver toutes les balises <a>
liens = soup.find_all("a")

# Extraire les URL des liens et les écrire dans un fichier
with open("urls_cuso.txt", "w") as f:
    for lien in liens:
        href = lien.get("href")
        url_complet = urljoin("http://cuso.ch", href)  # on peauffine les url
        f.write(url_complet + "\n")  # je saute une ligne entre les élesments avec "\n"


### Fini ??? (3) - pour plus tard

Si vous voulez aller plus loin en dehors de la correction de la séance :

Bah on peut toujours améliorer…

In [ ]:
# AUTRE LOGIQUE

# On aurait pu plutôt vouloir mettre tous les liens dans une liste
# Puis les sortir dans le fichier ensuite
# = une fois que j'ai ma liste je peux continuer a faire des opérations dessus

# importer
from bs4 import BeautifulSoup
import requests

# Récupérer la page
response = requests.get("http://cuso.ch")
soup = BeautifulSoup(response.content, "html.parser")

# Trouver toutes les balises <a>
liens = soup.find_all("a")

# Créer une liste contenant toutes les urls
liste_urls = []
for lien in liens:
    href = lien.get("href")  # on récupère le href
    url_complet = urljoin("http://cuso.ch", href)  # on peauffine les url
    liste_urls.append(url_complet)  # on les ajoute à notre liste

# Enregistrer la liste dans un fichier (un lien par ligne)
with open("toujours_plus_de_solutions.txt", "w", encoding="utf-8") as f:
    for u in liste_urls:
        f.write(u + "\n")

In [ ]:
### Puis améliorer

In [ ]:
# importer
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

# Récupérer la page
response = requests.get("http://cuso.ch")
soup = BeautifulSoup(response.content, "html.parser")

# Trouver toutes les balises <a>
liens = soup.find_all("a")

# Créer une liste contenant toutes les urls avec une list comprehension
liste_urls = [urljoin("http://cuso.ch", lien.get("href")) for lien in liens]

# Enregistrer la liste dans un fichier (un lien par ligne)
with open("encore_plus_de_solutions.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(liste_urls))


Et on gère toujours pas les erreurs possibles…

etc.